In [49]:
import requests
import pytz

import json
from datetime import datetime, time as datetime_time, timedelta
import calendar


In [8]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r


In [9]:
def make_api_request(stop_id, bearer):
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/OperationMonitoring/4.0/PassingTimeByPoint/" + stop_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()

In [10]:
def parse_waiting_times(json_payload, destination):
    passing_times = json_payload['points'][0]['passingTimes']
    str_expected_arrival_times = [passing_time['expectedArrivalTime'] for passing_time in passing_times if passing_time['destination']['fr']==destination]
    print(str_expected_arrival_times)
    expected_arrival_times = [datetime.fromisoformat(str_expected_arrival_time) for str_expected_arrival_time in str_expected_arrival_times]
    return expected_arrival_times 

In [11]:
def compute_time_diff(start, end):
    if isinstance(start, datetime_time): # convert to datetime
        assert isinstance(end, datetime_time)
        start, end = [datetime.combine(datetime.min, t) for t in [start, end]]
    if start <= end: # e.g., 10:33:26-11:15:49
        return end - start
    else: # end < start e.g., 23:55:00-00:25:00
        end += timedelta(1) # +day
        assert end > start
        return end - start

In [37]:
legrand_stop_id = "1046"
bearer = 'dd0508d9d488a5473fe2a594e12ad10a'
response = make_api_request(legrand_stop_id, bearer)
response

{'points': [{'passingTimes': [{'destination': {'fr': 'LOUISE', 'nl': 'LOUIZA'},
     'expectedArrivalTime': '2020-03-24T16:22:00+01:00',
     'lineId': '8'},
    {'destination': {'fr': 'LOUISE', 'nl': 'LOUIZA'},
     'expectedArrivalTime': '2020-03-24T16:34:00+01:00',
     'lineId': '8'}],
   'pointId': '1046'}]}

In [19]:
expected_arrival_times = parse_waiting_times(response, 'ROODEBEEK' )

current_time_utc = datetime.utcnow()
current_time_be = pytz.utc.localize(current_time_utc)
tram_arrival_time = expected_arrival_times[0]

waiting_time = compute_time_diff(current_time_be, tram_arrival_time)
print(waiting_time)


['2020-03-24T16:02:00+01:00', '2020-03-24T16:32:00+01:00']
0:10:18.639897


In [20]:
def convert_timedelta(duration):
    days, seconds = duration.days, duration.seconds
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = (seconds % 60)
    return days, hours, minutes, seconds

In [22]:
formatted_waiting_time = convert_timedelta(waiting_time)

In [23]:
formatted_waiting_time

(0, 0, 10, 18)

In [114]:
def compute_token_expiration_date(token_validity_time):
    now = datetime.utcnow()
    future = now + timedelta(seconds=token_validity_time)
    return calendar.timegm(future.utctimetuple())

In [115]:
token_expiration_date = compute_token_expiration_date(10)

In [116]:
def is_token_expired(token_expiration_date):
    current_time = datetime.now()
    unix_timestamp = current_time.timestamp()
    return unix_timestamp > token_expiration_date

In [117]:
test = is_token_expired(token_expiration_date)
test

False